# Running the 1000-genome workflow interactively on Kubernetes

To investigate *Jupyter-Workflow* strong scalability on a distribured infrastructure, we execute an 8-chromosomes instance of the [1000-genome workflow](https://github.com/pegasus-isi/1000genome-workflow) on up to 500 concurrent Kubernetes Pods. We selected the 1000-genome workflow for three main reasons:
* Pegasus is a state-of-the-art representative of HPC-oriented WMSs supporting execution environments without shared data spaces (via HTCondor);
* The host code of each step is written in either Bash or Python, both supported by the standard IPython kernel;
* The critical portion of the workflow id a highly-parallel step, composed of 2000 independent short tasks (~120s each on our infrastructure), which are critical for batch workload managers, but that can be executed at scale on on-demand Cloud resources (e.g. Kubernetes)

In order to replicate the experiment, you need a Kubernetes cluster with 3 control plane VMs (4 cores, 8GB RAM each) and 16 large worker VMs (40 cores, 120GB RAM each). In our infrastructure, each Kubernetes worker node has been manually placed on top of a different physical node, managed by an OpenStack Cloud controller. Nodes were interconnected by a 10Gbps Ethernet.

Each Pod requests 1 core and 2GB RAM and mounts a 1GB tmpfs under the `/tmp/streamflow` path, in order to avoid I/O bottlenecks. The description of such deployment is described in the `helm-1000-genome` model, which is managed by the StreamFlow Helm connector.

Initial inputs of the workflow are listed in this cell. Please note that, before launching the pipeline, you need to download all the input data. This can be easily done through the provided `data/download_data.sh` script.

In [0]:
import glob

chromosomes=[1, 2, 3, 4, 5, 6, 7, 8]
individuals=sorted(glob.glob("data/20130502/*.vcf.gz"))
annotations=sorted(glob.glob("data/20130502/sifting/*"))
populations = sorted(glob.glob("data/populations/*"))
columns = "data/20130502/columns.txt"
step = 1000

In the original Pegasus implementation, a chromosome input file is made available to all the workers involved in the *individuals* step in its entirety, and each of them selects a different partition to compute. Conversly, our implementation splits the dataset locally and then scatters it to the worker, transferring to each Pod only the strictly required amount of data.

This configuration is necessary to reach a decent level of performance in a distributed infrastructure, where the workers do not share a common data space. The original strategy would instead generate an unsustainable load on the communication infrastructure.

In [0]:
%%bash -s "$individuals" "$step" "$chromosomes"

inputfiles=($(echo $1 | tr -d "[],'"))
step="${2}"
chromosomes=($(echo $3 | tr -d "[],'"))

for i in ${!inputfiles[@]}; do
    inputfile=${inputfiles[i]}
    unzipped="${inputfile%.*z}"
    gunzip $inputfile

    split --lines "$step" --suffix-length 3 -d $unzipped ./"${unzipped##*/}.${chromosomes[i]}."
done

The aim of this step is to fetch and parse the Phase3 data from the 1000 genomes project by chromosome. These files list all of Single nucleotide polymorphisms (SNPs) variants in that chromosome and which individuals have each one. SNPs are the most common type of genetic variation among people, and are the ones we consider in this work. An individual task creates output files for each individual of rs numbers 3, where individuals have mutations on both alleles.

The first step, called *individuals*, is by far the heaviest in the workflow in terms of required computing time. Fortunately, it is also highly parallelizable. Indeed, each chromosome can be processed in parallel. Plus, each chromosome file is split in 250 chunks, for a total of 2000 independent tasks. Each task is then offloaded to one of the (up to 500) Pods in the `helm-1000-genome` model.

In [0]:
import time
stime = time.time()

In [0]:
%%bash -s "$individuals" "$columns"

inputfiles=($(echo $1 | tr -d "[],'"))
columfile="${2}"

for inputfile in ${inputfiles[*]}; do
    start=`date +%s.%N`
    basename="${inputfile##*/}"
    extension="${basename##*.}"
    nameroot="${basename%.*}"
    chromosome="${nameroot##*.}"
    outputfile="chr${chromosome}n-${extension}.tar.gz"
        
    pdir="chr${chromosome}p/"
    ndir="chr${chromosome}n/"
    mkdir -p "${pdir}" "${ndir}"
    
    
    if [ -s "${inputfile}" ]; then
        awk -v c=${chromosome} '{for(i=10;i<=2513;i++) {name="chr"c"p/chr"c".p"i-9;print $i"   "$2"    "$3"    "$4"    "$5"    "$8 >> name}}' ${inputfile}

        for i in {1..2504}; do
          col=$(($i + 9))
          name=$(cut -f ${col} ${columfile})
          oldfile="${pdir}chr${chromosome}.p${i}"
          newfile="${ndir}chr${chromosome}.${name}"
          cat ${oldfile} | awk '{print $6}' | awk -F ";" '{print $9}'| awk -F"=" '{print $2}' > "AF_value.${chromosome}"
          paste ${oldfile} AF_value.${chromosome} | awk '{$6=""; print}'> "tmp.file.${chromosome}"
          cat "tmp.file.${chromosome}" | awk ' $6 >= 0.5 {print $0}'| awk -F "|" '$1==0 || $2==0 {print $2}' > "tmp.select.${chromosome}"
          cat "tmp.file.${chromosome}" | awk ' $6 < 0.5 {print $0}'| awk -F "|" '$1==1 || $2==1 {print $2}' >> "tmp.select.${chromosome}"
          cat "tmp.select.${chromosome}" | awk '{print $2"        "$3"    "$4"    "$5"    "$6}' >> "${newfile}"
          rm ${oldfile}
        done

        rm "tmp.file.${chromosome}"
        rm "tmp.select.${chromosome}"
        rm "AF_value.${chromosome}"
    
        tar -czf "${outputfile}" -C "${ndir}" .
    else
        tar -czf "${outputfile}" -T "/dev/null"
    fi

    rm -rf "${ndir}"
    rm -rf "${pdir}"
    end=`date +%s.%N`
    echo "$(echo "$end $start" | awk '{print $1 - $2}')"
done

In [0]:
etime = time.time()
print(etime-stime)

1130.1189863681793


Each chromosome file is split in chunks to be processed in parallel by the previous step. Then, the results produced for each chunk of a chromosome must be merged (i.e. concatenated). Different chromosomes can still be processed in parallel. 

In [0]:
import posixpath
slices = list({k: [v for v in slices if posixpath.basename(v).startswith(k)] for k in ['chr{c}n'.format(c=c) for c in chromosomes]}.values())

In [0]:
%%bash -s "$slices"

start=`date +%s.%N`
slices=($(echo $1 | tr -d "[],'"))

for slice in ${slices[*]}; do
  echo "Merging ${slice}..."
  basename=${slice##*/}
  chromosome=${basename%-*}
  mkdir -p "merged-${chromosome}"
  mkdir -p tmp
  tar -xzf "${slice}" -C tmp
  if [ "$(ls -A tmp/)" ]; then
      for f in tmp/*; do
        cat "${f}" >> "merged-${chromosome}/${f##*/}"
      done
  fi
  rm -rf tmp
done

merged_folders=($(find . -name "merged-*"))

for merged_folder in ${merged_folders[*]}; do
    basename="${merged_folder##*/}"
    chromosome="${basename##*-}"
    outputfile="${chromosome}.tar.gz"
    echo "Creating merged file ${outputfile}..."
    tar -czf "${outputfile}" -C "${merged_folder}" .
    rm -rf "${merged_folder}"
done

end=`date +%s.%N`
echo "$(echo "$end $start" | awk '{print $1 - $2}')"

A *sifting* task computes the SIFT scores of all of the SNPs variants, as computed by the Variant Effect Predictor (VEP). SIFT is a sequence homology-based tool that Sorts Intolerant From Tolerant amino acid substitutions, and predicts whether an amino acid substitution in a protein will have a phenotypic effect.

For each chromosome the sifting task processes the corresponding VEP, and selects only the SNPs variants that has a SIFT score, recording in a file (per chromosome) the SIFT score and the SNPs variants ids, which are: (1) rs number, (2) ENSEMBL GEN ID, and (3) HGNC ID.

In [0]:
%%bash -s "$annotations" "$chromosomes"

inputfiles=($(echo $1 | tr -d "[],'"))
chromosomes=($(echo $2 | tr -d "[],'"))

for i in ${!inputfiles[@]}; do
    inputfile="${inputfiles[i]}"
    siftfile="ALL.chr'${chromosomes[i]}'.vcf.gz"
    unzipped="${inputfile%.*z}"
    gunzip "${inputfile}"
    header=$(head -n 1000 "${unzipped}" | grep "#" | wc -l)

    echo "taking columns from ${inputfile}"
    grep -n "deleterious\|tolerated" "${unzipped}" | grep "rs" > ${siftfile}

    lines='line.txt'
    ids='id.txt'
    info='info.txt'
    sifts='sift.txt'

    awk '{print $1}' ${siftfile} | awk -F ":" '{print $1-'${header}'}' > "${lines}"
    awk '{print $3}' ${siftfile} > ${ids}
    awk '{print $8}' ${siftfile} > ${info}
    awk -F "|" '{print $5"\t"$17"\t"$18}' "${info}" | sed 's/(/\t/g' | sed 's/)//g' > "${sifts}"

    final="sifted.SIFT.chr${chromosomes[i]}.txt"
    pr -m -t -s\ "${lines}" "${ids}" "${sifts}" | gawk '{print $1,$2,$3,$5,$7}' > "${final}"

    echo "line, id, ENSG id, SIFT, and phenotype printed to ${final}"

    rm "$siftfile" "$lines" "$ids" "$info" "$sifts" "$unzipped"
done

The following cells define some common Python code (imports and classes) that will be used by the last two steps of the workflow.

In the original setting there is a lot of duplicated code, due to the fact that each Python script must be self-contained, in order to be independently executed as an external command. Here we exploit the higher flexibility offered by the Jupyter Notebooks to refactor the code, removing duplicates.

In [0]:
import time

import numpy as np
import numpy.ma as ma
from random import sample
import os
import os.path
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib.pyplot import pcolor, show, colorbar, xticks, yticks
import matplotlib.cm as cm
from mpl_toolkits.axes_grid1 import make_axes_locatable
import itertools
import argparse
#import seaborn as sns
from matplotlib import pyplot
import matplotlib as mpl
import collections
from collections import Counter

SIFT = 'NO-SIFT'

In [0]:
class ReadData :
    def read_names(self, POP, columns) :
        print('reading inidviduals')
        f = open(POP, 'r')
        text = f.read()
        f.close()
        text = text.split()
        all_ids = text[0:]
        file = columns
        f = open(file, 'r')
        text = f.read()
        f.close()
        genome_ids = text.split()
        ids = list(set(all_ids) & set(genome_ids))
        return ids
    
    def read_rs_numbers(self, siftfile) :
        print('reading in rs with sift scores below %s' % SIFT)
        rs_numbers = []
        variations = {}
        map_variations = {}
        all_variations = []
        sift_file = open(siftfile,'r')
        for item in sift_file:
            item = item.split()
            if len(item) > 2:
                rs_numbers.append(item[1])
                map_variations[item[1]] = item[2]
        return rs_numbers, map_variations
    
    def read_individuals(self, ids, rs_numbers, chrom) :
        print('reading in individual mutation files')
        mutation_index_array = []
        total_mutations={}  
        total_mutations_list =[]    
        for name in ids :
            filename = './' + chrom + 'n/' + chrom + '.' + name
            f = open(filename, 'r')
            text = f.read()
            f.close()
            text = text.split()
            sifted_mutations = list(set(rs_numbers).intersection(text))
            mutation_index_array.append(sifted_mutations)
            total_mutations[name]= len(sifted_mutations)
            total_mutations_list.append(len(sifted_mutations))
        print ('mutation index array for %s : %s' % ( ids[0], mutation_index_array[0]))
        print ('total_len_mutations for %s : %s' % ( ids[0], total_mutations[ids[0]]))
        print('total_mutations_list is %s ' % total_mutations_list)
        return mutation_index_array, total_mutations, total_mutations_list    
   
    def read_pairs_overlap(self, indpairsfile) :
        print('reading in individual crossover mutations')
        pairs_overlap = np.loadtxt(indpairsfile, unpack=True)
        pairs_overlap = np.transpose(pairs_overlap)
        return pairs_overlap

In [0]:
class Results :
    def group_individuals(self, total_mutations_list, n_runs):
        print('histograms mutations_individuals groups by 26')
        n_group = 26
        random_mutations_list= []
        for run in range(n_runs):
            random_mutations_list.append(sample(total_mutations_list, n_group))
        return random_mutations_list

    def pair_individuals(self, mutation_index_array, n_runs) :
        print('cross matching mutations in individuals')
        n_p = len(mutation_index_array)
        n_pairs = int(round(n_p/2))
        list_p = np.linspace(0, n_p - 1, n_p).astype(int)
        pairs_overlap = np.zeros((n_runs, n_pairs))
        for run in range(n_runs) :
            randomized_list = sample(list(list_p) , n_p)
            for pq in range(n_pairs) :
                array1 = mutation_index_array[randomized_list[2*pq]]
                array2 = mutation_index_array[randomized_list[2*pq]]
                pair_array = set(array1) & set(array2)
                pairs_overlap[run][pq] = len(pair_array)
        return pairs_overlap

    def total_pair_individuals (self, mutation_index_array) :
        print('cross matching mutations total individuals')
        n_p = len(mutation_index_array)
        total_pairs_overlap = np.zeros((n_p, n_p))
        simetric_overlap = np.zeros((n_p, n_p))
        for run in range(n_p):
                        array1 = mutation_index_array[run]
                        start = run +1
                        for pq in range(start, n_p) :
                                array2 = mutation_index_array[pq]
                                pairs_array = set(array1) & set(array2)
                                total_pairs_overlap[run][pq]=len(pairs_array)
                                simetric_overlap[run][pq] = len(pairs_array)
                                simetric_overlap[pq][run]= len(pairs_array)
        return total_pairs_overlap , simetric_overlap

    def half_pair_individuals(self, mutation_index_array) :
        print('cross matching mutations in individuals - half with half')
        n_p = len(mutation_index_array)
        n_pairs = int(round(n_p/2))
        pairs_overlap = np.zeros((n_pairs, n_pairs))
        for run in range(n_pairs):
            array1 = mutation_index_array[run]
            index =0
            for pq in range(n_pairs+1, n_p):
                array2 = mutation_index_array[pq]
                pairs_array = set(array1) & set(array2)
                pairs_overlap[run][index]=len(pairs_array)
        return pairs_overlap

    def gene_pairs(self, mutation_index_array) :
        print('cross matching pairs of variations')
        n_p = len(mutation_index_array)
        gene_pair_list = {}
        for pp in range(n_p) :  
            pairs = itertools.combinations(mutation_index_array[pp], 2)
            for pair in pairs :
                key = str(pair)
                if key not in gene_pair_list : gene_pair_list[key] = 1
                else : gene_pair_list[key] += 1
        return gene_pair_list
    
    def overlap_ind(self, ids, mutation_index_array, n_runs, n_indiv):
        n_p = len(mutation_index_array)
        print('calculating the number overlapings mutations between %s individuals selected randomly' % n_p)
        list_p = np.linspace(0, n_p - 1, n_p).astype(int)
        mutation_overlap = []
        random_indiv = []
        for run in range(n_runs) :
            randomized_list = sample(list(list_p), n_p)
            result = Counter()
            r_ids=[]
            for pq in range(n_indiv):
                if 2*pq >= len(randomized_list):
                    break
                b_multiset = collections.Counter(mutation_index_array[randomized_list[2*pq]])
                print('time, inidividual: %s' % ids[randomized_list[2*pq]])
                r_ids.append(ids[randomized_list[2*pq]])
                result = result + b_multiset
            random_indiv.append(r_ids)
            mutation_overlap.append(result)
        return mutation_overlap, random_indiv
    
    def histogram_overlap(self, mutation_overlap, n_runs):
        print('calculating the frequency/historgram of overlapings mutations')
        histogram_overlap= []
        for run in range(n_runs):
            final_counts = [count for item, count in mutation_overlap[run].items()]
            histogram_overlap.append(collections.Counter(final_counts))
        return histogram_overlap

In [0]:
class PlotData :        
    def individual_overlap(self, POP, c, pairs_overlap, outputFile) :
        print('plotting cross matched number of individuals:%s '% len(pairs_overlap))
        pairs_overlap = np.array(pairs_overlap)     
        min_p = np.min(pairs_overlap)
        max_p = np.max(pairs_overlap)
        nbins = int(max_p) + 1
        n_runs = len(pairs_overlap)
        nbins = int(np.max(pairs_overlap))
        bin_centres = np.linspace(0, nbins, nbins)
        bin_edges = np.linspace(-0.5, nbins + 0.5, nbins + 1)
        fig = plt.figure(frameon=False, figsize=(10, 9))
        ax = fig.add_subplot(111)
        hists = []
        max_h = 0
        for run in range(n_runs) :
            h, edges = np.histogram(pairs_overlap[run], bins = bin_edges)
            ax.plot(bin_centres, h, alpha = 0.5)
            if len(h) > 0:
                max_h = max(max_h, max(h))
        plt.xlabel('Number of overlapping gene mutations', fontsize = 24)
        plt.ylabel(r'frequency', fontsize = 28)
        text1 = 'population ' + POP + '\n' +\
            'chromosome ' + str(c) + '\n' + \
            'SIFT < ' + str(SIFT) + '\n' + \
            str(n_runs) + ' runs'
        plt.text(.95, .95, text1, fontsize = 24, 
            verticalalignment='top', horizontalalignment='right',
            transform = ax.transAxes)
        plt.savefig(outputFile)  
        plt.close()

    def total_colormap_overlap(self, POP, total_pairs_overlap, outputFile):
        print('plotting colormap number of individuals: %s' % len(total_pairs_overlap))
        fig = plt.figure()
        cmap = mpl.colors.ListedColormap(['blue','black','red', 'green', 'pink'])
        img = pyplot.imshow(total_pairs_overlap,interpolation='nearest', cmap = cmap, origin='lower')
        pyplot.colorbar(img,cmap=cmap)
        plt.savefig(outputFile)  
        plt.close()
    
    def plot_histogram_overlap(self, POP, histogram_overlap, outputFile, n_runs):
        print('ploting Histogram mutation overlap to %s' % outputFile)
        for run in range(n_runs):
            output = outputFile + str(run) + '.png'
            final_counts = [count for item, count in histogram_overlap[run].items()]
            N = len( final_counts )
            x = range( N )
            width = 1/1.5
            bar1=plt.bar( x, final_counts, width, color="grey" )
            plt.ylabel( 'Mutations' )
            plt.xlabel('Individuals')
            plt.xticks( np.arange( 1,N+1 ) )
            plt.savefig(output)
            plt.close()

In [0]:
class WriteData :
    def write_pair_individuals(self, indpairsfile, pairs_overlap) : 
        print('writing pairs overlapping mutations to %s' % indpairsfile)
        np.savetxt(indpairsfile, pairs_overlap, fmt = '%i')
    
    def write_gene_pairs(self, genepairsfile, gene_pair_list) :
        print('writing gene pair list to %s'% genepairsfile)
        f = open(genepairsfile, 'w')
        for key, count in gene_pair_list.items() :
            f.write(key + '\t' + str(count) + '\n')
        f.close()
    
    def write_total_indiv(self, total_mutations_filename, total_mutations) :
        print('writing total mutations list per individual to %s' % total_mutations_filename)
        f = open(total_mutations_filename, 'w')
        for key, count in total_mutations.items() :
            f.write(key + '\t' + str(count) + '\n')
        f.close()

    def write_random_indiv(self, randomindiv_file, random_indiv, n_runs) :
        for run in range(n_runs):
            randomfile = randomindiv_file + str(run) + '.txt'
            f = open(randomfile, 'w')
            print('writing Random individuals to %s' % randomfile)
            f.write('Individuals \n')
            for item in random_indiv[run]:
                f.write("%s\n" % item)
            f.close()
        
    def write_random_mutations_list(self, random_mutations_filename, random_mutations_list, n_runs) :
        print('writing a list of 26 random individuals with the number mutations per indiv %s' % random_mutations_filename)
        for run in range(n_runs):
            filename= random_mutations_filename +'_run_' + str(run) + '.txt'
            f = open(filename, 'w')
            f.writelines(["%s\n" % item  for item in random_mutations_list[run]])
    
    def write_mutation_index_array(self, mutation_index_array_file, mutation_index_array):
        print('writing mutation array  to %s' % mutation_index_array_file)
        f=open(mutation_index_array_file,"w")
        for item in mutation_index_array:
            f.write("%s\n" % item)
        f.close()

    def write_map_variations(self, map_variations_file, map_variations) :
        print('writing map_variations to %s' % map_variations_file)
        f = open(map_variations_file, 'w')
        for key, count in map_variations.items() :
            f.write(key + '\t' + str(count) + '\n')
        f.close()
        
    def write_histogram_overlap(self, histogram_overlapfile, histogram_overlap, n_runs, n_indiv) :
        print('writing Frequency historgram of mutations overlapping to %s' % histogram_overlapfile)
        for run in range(n_runs):
            overlapfile = histogram_overlapfile + str(run) + '.txt'
            f = open(overlapfile, 'w')
            f.write('Number Individuals - Number Mutations  \n')
            for i in range(1,n_indiv+1):
                if i in histogram_overlap[run]:
                    f.write(str(i) + '-' + str(histogram_overlap[run][i]) + '\n')
                else:
                    f.write(str(i) + '-' + str(0) + '\n')
            f.close()
            
    def write_mutation_overlap(self, mutation_overlapfile, mutation_overlap, n_runs) :
        print('writing Mutations overlapping to %s' % mutation_overlapfile)
        for run in range(n_runs):
            overlapfile = mutation_overlapfile + str(run) + '.txt'
            f = open(overlapfile, 'w')
            f.write('Mutation Index- Number Overlapings \n')
            for key, count in mutation_overlap[run].items() :
                f.write(key + '-' + str(count) + '\n')
            f.close()

This task measures the overlap in mutations (also called SNPs variants) among pairs of individuals by population and by chromosome.

In [ ]:
import tarfile

n_runs = 1
outdata_dir = './output_no_sift/'
plot_dir = './plots_no_sift/'
OutputFormat = '.png'

if not os.path.exists(outdata_dir):
    os.makedirs(outdata_dir)
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)

for i, c in enumerate(chromosomes):
    
    tar = tarfile.open(merged[i])
    tar.extractall(path='./chr' + str(c) + 'n')
    tar.close()
    
    for pop in populations:
        font = {'family':'serif', 'size':14}
        plt.rc('font', **font)

        rd = ReadData()
        res = Results()
        wr = WriteData()
        pd = PlotData()
        
        POP = os.path.basename(pop)
        half_indpairsfile = outdata_dir + 'individual_half_pairs_overlap_chr' + str(c) + '_s' + \
            str(SIFT) + '_' + POP + '.txt'
        total_indpairsfile = outdata_dir + 'total_individual_pairs_overlap_chr' + str(c) + '_s' + \
            str(SIFT) + '_' + POP + '.txt'
        genepairsfile = outdata_dir + 'gene_pairs_count_chr' + str(c) + '_s' + \
            str(SIFT) + '_' + POP + '.txt'
        random_indpairsfile = outdata_dir + '100_individual_overlap_chr' + str(c) + '_s' + \
            str(SIFT) + '_' + POP + '.txt'

        colormap = plot_dir + 'colormap_distribution_c' + str(c) + '_s' + \
                str(SIFT) + '_' + POP + OutputFormat
        half_overlap = plot_dir + 'half_distribution_c' + str(c) + '_s' + \
                str(SIFT) + '_' + POP + OutputFormat
        total_overlap = plot_dir + 'total_distribution_c' + str(c) + '_s' + \
                str(SIFT) + '_' + POP + OutputFormat
        random_overlap = plot_dir + '100_distribution_c' + str(c) + '_s' + \
                str(SIFT) + '_' + POP + OutputFormat

        total_mutations_filename = outdata_dir + 'total_mutations_individual' + str(c) + '_s' + \
            str(SIFT) + '_' + POP + '.txt'
        random_mutations_filename = outdata_dir + 'random_mutations_individual' + str(c) + '_s' + \
            str(SIFT) + '_' + POP 

        mutation_index_array_file = outdata_dir + 'mutation_index_array' + str(c) + '_s' + \
            str(SIFT) + '_' + POP + '.txt'

        map_variations_file = outdata_dir + 'map_variations' + str(c) + '_s' + \
            str(SIFT) + '_' + POP + '.txt'

        ids = rd.read_names(pop, columns)
        n_pairs = len(ids)/2


        rs_numbers, map_variations = rd.read_rs_numbers(sift[i])
        mutation_index_array, total_mutations, total_mutations_list = rd.read_individuals(ids, rs_numbers, 'chr' + str(c))
        wr.write_total_indiv(total_mutations_filename, total_mutations)
        wr.write_map_variations(map_variations_file, map_variations)

        #cross-correlations mutations overlapping
        half_pairs_overlap = res.half_pair_individuals(mutation_index_array)
        total_pairs_overlap, simetric_overlap = res.total_pair_individuals(mutation_index_array)
        random_pairs_overlap = res.pair_individuals(mutation_index_array, n_runs)

        wr.write_mutation_index_array(mutation_index_array_file, mutation_index_array)
        wr.write_pair_individuals(half_indpairsfile, half_pairs_overlap)
        wr.write_pair_individuals(total_indpairsfile, total_pairs_overlap)
        wr.write_pair_individuals(random_indpairsfile, random_pairs_overlap)

        pd.individual_overlap(POP, c, half_pairs_overlap, half_overlap)
        pd.individual_overlap(POP, c, simetric_overlap, total_overlap)
        pd.individual_overlap(POP, c, random_pairs_overlap, random_overlap)
        pd.total_colormap_overlap(POP, total_pairs_overlap, colormap)

        #list of frecuency of mutations in 26 individuals
        random_mutations_list=res.group_individuals(total_mutations_list, n_runs)
        wr.write_random_mutations_list(random_mutations_filename, random_mutations_list, n_runs)

        # gen overlapping
        gene_pair_list = res.gene_pairs(mutation_index_array)
        wr.write_gene_pairs(genepairsfile, gene_pair_list)

        # gen final output
        tar = tarfile.open('chr%s-%s.tar.gz' % (c, POP), 'w:gz')
        tar.add(outdata_dir)
        tar.add(plot_dir)
        tar.close()

This tasks measures the frequency of overlapping in mutations by selecting a number of random individuals, and selecting all SNPs variants without taking into account their SIFT scores.

In [ ]:
import tarfile

n_runs = 1000
n_indiv= 52
outdata_dir = './output_no_sift/'
plot_dir = './plots_no_sift/'

OutputFormat = '.png'

if not os.path.exists(outdata_dir):
    os.makedirs(outdata_dir)
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)
    
for i, c in enumerate(chromosomes):
    
    tar = tarfile.open(merged[i])
    tar.extractall(path='./chr' + str(c) + 'n')
    tar.close()
    
    for pop in populations:
        font = {'family':'serif', 'size':14}
        plt.rc('font', **font)

        rd = ReadData()
        res = Results()
        wr = WriteData()
        pd = PlotData()
        
        POP = os.path.basename(pop)
        histogram_overlapfile = outdata_dir + 'Histogram_mutation_overlap_chr' + str(c) + '_s' + \
            str(SIFT) + '_' + POP + '_'
        mutation_overlapfile = outdata_dir + 'Mutation_overlap_chr' + str(c) + '_s' + \
            str(SIFT) + '_' + POP + '_'
        mutation_index_array_file = outdata_dir + 'mutation_index_array' + str(c) + '_s' + \
            str(SIFT) + '_' + POP + '.txt'
        histogram_overlap_plot = plot_dir + 'Frequency_mutations' + str(c) + '_s' + \
            str(SIFT) + '_' + POP 
        map_variations_file = outdata_dir + 'map_variations' + str(c) + '_s' + \
            str(SIFT) + '_' + POP + '.txt'
        randomindiv_file = outdata_dir + 'random_indiv' + str(c) + '_s' + \
            str(SIFT) + '_' + POP + '_'

        ids = rd.read_names(pop, columns)
        n_pairs = len(ids)/2

        rs_numbers, map_variations = rd.read_rs_numbers(sift[i])
        mutation_index_array, _, _ = rd.read_individuals(ids, rs_numbers, 'chr' + str(c))
        print(mutation_index_array)

        wr.write_map_variations(map_variations_file, map_variations)
        wr.write_mutation_index_array(mutation_index_array_file, mutation_index_array)

        mutation_overlap, random_indiv= res.overlap_ind(ids, mutation_index_array, n_runs, n_indiv)
        histogram_overlap= res.histogram_overlap(mutation_overlap, n_runs)

        wr.write_mutation_overlap(mutation_overlapfile, mutation_overlap, n_runs)
        wr.write_histogram_overlap(histogram_overlapfile, histogram_overlap, n_runs, n_indiv)
        wr.write_random_indiv(randomindiv_file, random_indiv, n_runs)

        pd.plot_histogram_overlap(POP, histogram_overlap, histogram_overlap_plot, n_runs)

        # gen final output
        tar = tarfile.open('chr%s-%s-freq.tar.gz' % (c, POP), 'w:gz')
        tar.add(outdata_dir)
        tar.add(plot_dir)
        tar.close()